In [1]:
import pandas as pd

# Load excel
df = pd.read_excel("portfolio.xlsx")

# Forward fill Date Range column
df['Date Range'] = df['Date Range'].ffill()

# Extract start date
df['Start Date'] = df['Date Range'].str.split(" to ").str[0]
df['Start Date'] = pd.to_datetime(df['Start Date'])

# Sort
df = df.sort_values(['Constituents', 'Start Date'])

In [2]:
df.head(10)

,Date Range,Constituents,Weightage,Start Date
1311,2021-11-08 to 2021-11-14,ACC Ltd,0.08,2021-11-08
1321,2021-11-15 to 2021-11-21,ACC Ltd,0.08,2021-11-15
1331,2021-11-22 to 2021-11-28,ACC Ltd,0.08,2021-11-22
2233,2023-10-16 to 2023-10-18,APL Apollo Tubes Ltd,0.09,2023-10-16
81,2019-05-27 to 2019-06-02,AU Small Finance Bank Ltd,0.10,2019-05-27
91,2019-06-03 to 2019-06-09,AU Small Finance Bank Ltd,0.10,2019-06-03
100,2019-06-10 to 2019-06-16,AU Small Finance Bank Ltd,0.10,2019-06-10
111,2019-06-17 to 2019-06-23,AU Small Finance Bank Ltd,0.10,2019-06-17
120,2019-06-24 to 2019-06-30,AU Small Finance Bank Ltd,0.11,2019-06-24
131,2019-07-01 to 2019-07-07,AU Small Finance Bank Ltd,0.11,2019-07-01


In [3]:

holding_periods = []

for stock, group in df.groupby('Constituents'):
    group = group.sort_values('Start Date')
    dates = group['Start Date'].unique()

    entry = dates[0]
    prev_date = dates[0]

    for current_date in dates[1:]:
        if (current_date - prev_date).days > 7:
            # Exit detected
            holding_periods.append({
                "Stock": stock,
                "Entry": entry,
                "Exit": prev_date
            })
            entry = current_date

        prev_date = current_date

    # Final exit
    holding_periods.append({
        "Stock": stock,
        "Entry": entry,
        "Exit": prev_date
    })

holding_df = pd.DataFrame(holding_periods)

holding_df['Holding Days'] = (holding_df['Exit'] - holding_df['Entry']).dt.days
holding_df['Holding Weeks'] = holding_df['Holding Days'] / 7


In [4]:
holding_df.head(20)

,Stock,Entry,Exit,Holding Days,Holding Weeks
0,ACC Ltd,2021-11-08,2021-11-22,14,2.000000
1,APL Apollo Tubes Ltd,2023-10-16,2023-10-16,0,0.000000
2,AU Small Finance Bank Ltd,2019-05-27,2019-07-08,42,6.000000
3,Aarti Drugs Ltd,2020-05-18,2020-05-18,0,0.000000
4,Aarti Drugs Ltd,2020-05-26,2020-10-19,146,20.857143
5,Adani Enterprises Ltd,2022-07-11,2022-08-08,28,4.000000
6,Adani Enterprises Ltd,2022-08-16,2022-10-03,48,6.857143
7,Adani Green Energy Ltd,2020-05-18,2020-05-18,0,0.000000
8,Adani Green Energy Ltd,2020-05-26,2020-11-09,167,23.857143
9,Adani Green Energy Ltd,2020-11-17,2020-11-23,6,0.857143


In [5]:

import plotly.express as px

fig = px.timeline(
    holding_df,
    x_start="Entry",
    x_end="Exit",
    y="Stock",
    color="Stock"
)

fig.update_yaxes(autorange="reversed")
fig.show()
